In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py, time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mat
import seaborn as sb
import tensorflow as tf
import scipy

import sys
sys.path.append('../../../..')
import mutagenesisfunctions as mf
import bpdev as bd
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

from Bio import AlignIO
import time as time
import pandas as pd
np.random.seed(42)

In [15]:
os.getcwd()

'/home/peter/steffan/rna_secstructure/rna_secstructure_code/marks_figures/R-scape validation'

# Move sorted.out files to rscape_scores

In [17]:
fam_done = ['RF0005', 'RF00010', 'RF00023', 'RF00059']
families = ['RF00002', 'RF00005', 'RF00010', 'RF00017', 'RF00023', 'RF00050',
            'RF00059', 'RF00162', 'RF00167', 'RF00169', 'RF00174', 'RF00234',
            'RF00380', 'RF00504', 'RF01734', 'RF01786', 'RF01831', 'RF01852', 'RF02001']#, 'RF01960']

from shutil import copyfile

for fam in families:
    if fam not in fam_done:
        copyfile('../../../data_marks/%s/rtest/%s_red.sorted.out'%(fam,fam), 'rscape_scores/%s_red.sorted.out'%(fam))

# Extract scores in a txt legible for pandas

In [18]:
for fam in families:
    if fam not in fam_done:

        routfile = '%s_red.sorted.out'%(fam)
        ritefile = '%s_rscores.txt'%(fam)

        #extract the lines we want from the outfile
        ritelines = []
        with open(os.path.join('rscape_scores', routfile), 'r') as fd:
            start = False
            done = False
            for line in fd:
                if not start:
                    if 'left_pos' in line:
                        start = True
                        ritelines.append(line)
                    else:
                        continue
                elif start:
                    if 'cyk-cov' in line:
                        done = True
                    else:
                        ritelines.append(line)

                if done:
                    break

        #write them to a temp file
        with open(os.path.join('rscape_scores', ritefile), 'w') as fd:
            for line in ritelines:
                fd.write(line)

In [ ]:
#read it in as a csv
rscores = pd.read_csv(os.path.join('rscape_scores', ritefile), delim_whitespace=True).iloc[1:, 1:]
rscores